# 第３章: [正規表現](https://nlp100.github.io/ja/ch03.html)

[脇田の回答例](https://colab.research.google.com/drive/12ejBdUYMdvgoBi815E1AZpbrfsUQ9kki?usp=sharing)

# 20. JSONデータの読み込み

何度もファイルをダウンロードしないように `os.path.exists` で存在しないことを確認してから、ダウンロードする。

In [4]:
import gzip
import os, os.path

if not os.path.exists('jawiki-country.json.gz'):
    os.system('wget --quiet https://nlp100.github.io/data/jawiki-country.json.gz')

つぎにダウンロードしたファイルを読み込み、イギリスのデータを取得し、`uk_text` という名前の変数に保存した。

- `GZip` で圧縮されたファイルを開くには、[`gzip.open` 関数](https://docs.python.org/3.10/library/gzip.html#gzip.open) を使う。使い方は普通の [`open`関数](https://docs.python.org/3.10/library/functions.html#open) とほぼ同じ。

- JSON形式の文字列を読み込むには[`json.loads`関数](https://docs.python.org/3.10/library/json.html#json.loads)を使う。ファイルハンドルから読み込むときは [`json.load`関数](https://docs.python.org/3.10/library/json.html#json.load)、逆にファイルハンドル (`w`) に書き込むときは[`json.dump(w, json_data)`関数](https://docs.python.org/3.10/library/json.html#json.dump)。JSON 形式の文字列を取得するには [`json.dumps`関数](https://docs.python.org/3.10/library/json.html#json.dumps)を使う。

In [6]:
import json

with gzip.open('jawiki-country.json.gz') as gz:
    while True:
        input = gz.readline()
        if input == '': break
        data = json.loads(input)
        if data['title'] == 'イギリス':
            uk_text = data['text'].split('\n')
            break

# 21. カテゴリ名を含む行を抽出

この章の残りの問題を解くヒントを得るためにデータ形式を解析するために使う関数を定義する。

In [7]:
import re

def grep(pat):
    '''uk_text の行のうち pat に合致するものを出力する。
       Input:
         - pat: 正規表現
    '''
    rx = re.compile(pat)
    for line in uk_text:
        if rx.search(line): print(line)

grep('Category')

{{Sisterlinks|commons=United Kingdom|commonscat=United Kingdom|s=Category:イギリス|n=Category:イギリス|voy=United Kingdom}}
[[Category:イギリス|*]]
[[Category:イギリス連邦加盟国]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国|元]]
[[Category:海洋国家]]
[[Category:現存する君主国]]
[[Category:島国]]
[[Category:1801年に成立した国家・領域]]


# 22. カテゴリ名の抽出

ここから本格的な正規表現の練習が始まる。詳しくは [Regular Expression HOWTO](https://docs.python.org/3.10/howto/regex.html) を参照のこと。

よく使う正規表現のメタ記号の使い方は：

- `[abcdef]`: `a` or `b` or `c` or `d` or `e`

- `[a-z]`: `a` or `b` or ... or `y` or `z`。`[A-Z]` や `[0-9]` も同様

- `[^abcdef]`: `[a-f]` に適合しない文字

- `[^]abc]`: `]`, `a`, `b`, `c` 以外の文字

- `X|Y`: `X` あるいは `Y` のいずれかの正規表現に適合する文字列

- `X*`: 正規表現`X`に適合する文字列の0個以上を接続したもの

- `X+`: 正規表現`X`に適合する文字列の1個以上を接続したもの

- `X(Y)Z`: 正規表現`XYZ`に適合する文字列。この正規表現の `match` メソッドが利用された場合、括弧に囲われた `Y` に適合する文字列を簡単に取得できる。

    ~~~
    rx = re.compile('-+ ([^-]+) -+')
    m = rx.match('---------- This is an important message ---')
    
    if m: print(m[1])  # You see: This is an important message
    ~~~

この章の課題は多くは以下のような形式で解けると思う。

~~~
rx = re.compile('グループ機能を用いた正規表現')

for line in uk_text:
    m = rx.match(line)
    if m: print(m[1])  # m[1] は正規表現の1番目のグループに適合した部分文字列
~~~

# 23. セクション構造

[MediaWiki のセクション構造](https://www.mediawiki.org/wiki/Help:Formatting#Level_2)は二個以上の `==` でセクションタイトルを挟んで以下のように記述するらしい。

~~~
== Level 2 ==

=== Level 3 ===

==== Level 4 ====

===== Level 5 =====

====== Level 6 ======
~~~

これを正規表現で記述すればよい。ところで、上の例はタイトルを挟んで左右同数の `=` が出現している。Pumping 補題より、このようなパターンを正規表現で記述することはできないことが知られている。正規表現で記述する以上、同数条件は記述しないでよい。

脇田の回答では、たくさん出力される項目を `IPython.display.Markdown` を用いて、表形式で出力している。

# 24. ファイル参照の抽出

以下を実行して考える。

~~~
grep('ファイル')
~~~

# 25. テンプレートの抽出

以下を実行して取り組む。出力をよくよくよく観察すること。

~~~
grep('基礎情報')
~~~

# 26. 強調マークアップの除去

`"確立形態4 ": "現在の国号「'''グレートブリテン及び北アイルランド連合王国'''」に変更"`が該当するらしい。

# 27. 内部リンクの除去

似たようなコードを書くのが面倒になってくるころかと思う。きれいなプログラムを書くには工夫を要する。

# 28. MediaWikiマークアップの除去

正規表現についての最後の課題だ。がんばれ！

# 29. 国旗画像のURLを取得する

まず、言われたとおりに [MediaWiki の `imageinfo` API のドキュメント](https://www.mediawiki.org/wiki/API:Imageinfo)を眺めてみる。よくわからなくても、あきらめずに我慢して読む。

説明に続いて、API を利用したときに返ってくる JSON データ (**Response**) と Python のサンプルコード (**Sample Code**) が掲載されている。

~~~
import requests

S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

PARAMS = {
    "action": "query",
    "format": "json",
    "prop": "imageinfo",
    "titles": "File:Billy_Tipton.jpg"
}

R = S.get(url=URL, params=PARAMS)
DATA = R.json()

PAGES = DATA["query"]["pages"]

for k, v in PAGES.items():
    print(v["title"] + " is uploaded by User:" + v["imageinfo"][0]["user"])
~~~

**Sample code** の `PAGES` 変数に代入されている `DATA["query"]["pages"]` が、**Response** の構造を反映していることに注目。`PAGES` は `batchcomplete`, `query` 属性を持つJSON 辞書で、`query` 属性の値もまた JSON 辞書。`query` 属性の値の `pages` 属性の値を `PAGES` に代入していることになる。`for k, v in PAGES.items():` のなかで `title`, `imageinfo` などを取得しているが、それと `PAGES` の構造をよく見比べること。

この例は `Billy_Tipton.jpg` という画像の `title` を取得する例だった。画像を取得するにはどうすればよいだろうか？もう一度、[MediaWiki の `imageinfo` API のドキュメント](https://www.mediawiki.org/wiki/API:Imageinfo) を読み返せばわかるだろう。